<a href="https://colab.research.google.com/github/b-fatma/S2I-DM/blob/master/src/3_merge/v2/merge_0_fire_landcover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import geopandas as gpd
import pandas as pd
import csv
import sys
import geopandas as gpd
import shapely

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
csv.field_size_limit(sys.maxsize)

131072

## Reading fire data

In [4]:
input_path = '/content/drive/MyDrive/dm_fire_prediction/premerge/fire.csv'

df = pd.read_csv(input_path)
df.head()

,longitude,latitude,fire
0,5.53337,35.70751,1
1,6.46961,32.13579,1
2,6.97630,32.35563,1
3,9.39581,28.19791,1
4,9.49323,28.12826,1


In [5]:
input_path_lc = '/content/drive/MyDrive/dm_fire_prediction/premerge/landcover.csv'
landcover_df = pd.read_csv(input_path_lc)
landcover_df.head()

,GRIDCODE,geometry
0,210,"POLYGON ((6.415278 37.086959, 6.431034 37.0855..."
1,210,"POLYGON ((7.180841 37.079167, 7.179981 37.0809..."
2,210,"POLYGON ((7.371375 37.081944, 7.370904 37.0871..."
3,50,"POLYGON ((6.123611 36.684722, 6.123611 36.6930..."
4,210,"POLYGON ((6.261815 37.023611, 6.261927 37.0251..."


In [7]:
landcover_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438513 entries, 0 to 438512
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   GRIDCODE  438513 non-null  int64 
 1   geometry  438513 non-null  object
dtypes: int64(1), object(1)
memory usage: 6.7+ MB


In [8]:
gdf_land = landcover_df.copy()
gdf_land["geometry"] = shapely.from_wkt(gdf_land.geometry)
gdf_land = gpd.GeoDataFrame(gdf_land, geometry="geometry", crs="EPSG:4326")

In [9]:
fire_df = df.copy()

In [10]:
from shapely.geometry import Point

gdf_fire = gpd.GeoDataFrame(
    fire_df,
    geometry=[Point(xy) for xy in zip(fire_df.longitude, fire_df.latitude)],
    crs="EPSG:4326"
)

In [11]:
result = gpd.sjoin(
    gdf_fire,
    gdf_land[["geometry", "GRIDCODE"]],
    how="left",
    predicate="intersects"
)


In [12]:
result = result.drop(columns="index_right")

In [13]:
result

,longitude,latitude,fire,geometry,GRIDCODE
0,5.533370,35.707510,1,POINT (5.53337 35.70751),134.0
1,6.469610,32.135790,1,POINT (6.46961 32.13579),202.0
2,6.976300,32.355630,1,POINT (6.9763 32.35563),202.0
3,9.395810,28.197910,1,POINT (9.39581 28.19791),201.0
4,9.493230,28.128260,1,POINT (9.49323 28.12826),201.0
...,...,...,...,...,...
1965251,9.825798,37.320199,0,POINT (9.8258 37.3202),30.0
1965252,9.834781,37.320199,0,POINT (9.83478 37.3202),30.0
1965253,9.843764,37.320199,0,POINT (9.84376 37.3202),30.0
1965254,9.852747,37.320199,0,POINT (9.85275 37.3202),30.0


In [14]:
result.drop(columns='geometry')

,longitude,latitude,fire,GRIDCODE
0,5.533370,35.707510,1,134.0
1,6.469610,32.135790,1,202.0
2,6.976300,32.355630,1,202.0
3,9.395810,28.197910,1,201.0
4,9.493230,28.128260,1,201.0
...,...,...,...,...
1965251,9.825798,37.320199,0,30.0
1965252,9.834781,37.320199,0,30.0
1965253,9.843764,37.320199,0,30.0
1965254,9.852747,37.320199,0,30.0


In [15]:
result[result['GRIDCODE'].isna()][result['fire'] == 1]

/usr/local/lib/python3.12/dist-packages/geopandas/geodataframe.py:1896: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


,longitude,latitude,fire,geometry,GRIDCODE
3240,3.07880,36.75200,1,POINT (3.0788 36.752),NaN
6496,6.56613,37.01773,1,POINT (6.56613 37.01773),NaN
6523,6.56692,37.02195,1,POINT (6.56692 37.02195),NaN
8773,3.07719,36.75280,1,POINT (3.07719 36.7528),NaN
9292,5.07765,36.74498,1,POINT (5.07765 36.74498),NaN


In [16]:
output_path = '/content/drive/MyDrive/dm_fire_prediction/mergee/fire_landcover.csv'
result.to_csv(output_path, index=False)